# Testing Model
## Importing libraries

In [ ]:
import os
from PIL import Image
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import model_from_json
import os
import sys
import pickle
import camera
!pip3 install --ignore-installed h5py
import h5py

## Defining Classes for 43 Traffic Signs

In [8]:
labels=['Speed Limit 20', 'Speed Limit 30', 'Speed Limit 50', 'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', '', 'Speed Limit 100', 'Speed Limit 120', 'No Passing', 'No Passing by heavy Vehicles', 'Priority at next intersection', 'Priority Road', 'Give Way', 'STOP', 'No Entry in Both Directions', 'Traffic Sign 1', 'No Entry', 'Caution sign', 'Dangerous left curve',
'Dangerous Right Curve', 'Left Reverse Bend', 'Uneven Road', 'Slippery Road', 'Right Lane Ends', 'Men At Work', 'Traffic Light Ahead', 'Traffic Sign 2', 'Traffic Sign 3', 'Cycle Lane', 'Risk Of Ice Snowdrift', 'Deer Crossing Warning', 'Traffic Sign 4', 'Right Turn', 'Left Turn', 'Straight Ahead', 'Traffic Sign 5', 'Traffic Sign 6', 'Traffic Sign 7', 'Traffic Sign 8', 'Roundabout', 'Traffic Sign 9', 'Traffic Sign 10']

## Loading model design and weights from pre-saved files

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
sys.path.append('/usr/local/lib/python3.5/dist-packages')
print(sys.path)
model.load_weights("model.h5")
print("Loaded model from disk")

## Compiling model and displaying model summary

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## Loading Testing/Validation Images and corresponding labels to which they belong to.
### Converting images to corresponding numpy arrays.
#### Labels range from 0-42.

In [ ]:
testing_labels = './Labels/test.p'
with open(testing_labels, mode='rb') as f:
    test = pickle.load(f)
images_test = []
temp_img_test = []
for img in dir_test:
    img=str(img)
    allowed_extension = 'ppm'
    if (img.split('.'))[1]==allowed_extension:
        path =url + '/{}'.format(img)
        im = Image.open(path)
        im = im.convert('L')
        im = im.resize((64, 64), Image.ANTIALIAS)
        im = np.array(im)
        temp_img_test.append(im)
images_test = np.asarray(temp_img_test)

## Scaling values of numpy arrays(obtained in previous step) between 0 an 1.

In [ ]:
images_test = images_test/255
images_test = images_test.reshape(images_test.shape[0], 64, 64, 1)
images_test = images_test.astype('float32')

## Evaluating model on the test set and computing Test Accuracy

In [ ]:
score = model.evaluate(np.array(images_test), np.array(y_test), verbose=0)
print('Test score: ', score[0])
print('Test accuracy: ', score[1])

## 2 options for user to either test the image on already provided sample image or to capture own image of a traffic sign

In [ ]:
print('Do you want to capture your own image(1) or want to test it on a custom image(2). Choose 1 or 2.')
entry=int(input())
if entry==2:
    im = Image.open('./sample.ppm')
    im = im.convert('L')
    im = im.resize((64,64), Image.ANTIALIAS)
    im = np.array(im)
    x = np.expand_dims(im, axis=0)
    x=x.reshape(1,64,64,1)

    out1 = model.predict(x)
    y = np.argmax(out1)
    print(labels[y])
elif entry == 1:
    camera.capture()
    im = Image.open('./captured_sign.png')
    im = im.convert('L')
    im = im.resize((64,64), Image.ANTIALIAS)
    im = np.array(im)
    x = np.expand_dims(im, axis=0)
    x=x.reshape(1,64,64,1)

    out1 = model.predict(x)
    y = np.argmax(out1)
    print(labels[y])
    os.remove('./captured_sign.png')
else:
    print('Please choose the correct option and reexecute the code.')